In [2]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
# Load environment variables from .env
load_dotenv()



True

In [3]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("what is the main purpose of whatfix company ")
print(response.content)

Whatfix is a digital adoption platform (DAP) company that provides a suite of tools to help organizations improve the onboarding, training, and support of their employees, customers, or users on various software applications, websites, and digital products.

The main purpose of Whatfix is to:

1. **Simplify complex software applications**: Whatfix helps to break down complex software applications into simple, easy-to-follow instructions, making it easier for users to learn and use them.
2. **Improve user adoption and engagement**: By providing contextual, in-app guidance and support, Whatfix enables users to quickly get up to speed with new software applications, reducing the time it takes to become proficient.
3. **Reduce support queries and tickets**: Whatfix's interactive guides and tutorials help users to resolve common issues and answer frequently asked questions, reducing the number of support queries and tickets.
4. **Enhance overall user experience**: By providing personalized,

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://medusajs.com/careers/")
page_data = loader.load().pop().page_content
print(page_data)

Medusa - Join UsProductSolutionsDevelopersPricingWe're hiringDocsSign up MenuDocs·Sign upCareersChange how
commerce is builtBuilding a generational company requires exceptional, hard-working people. We are tackling the complexities of commerce infrastructure that no one else has dared to.Join the teamSebastian RindomOliver JuhlNicklas Gellner+15Engineering·Europe & US East CoastSoftware Engineer, Open-sourceAs apart of our open-source team, you will help improve our commerce platform and framework tooling to benefit Medusa developers and merchants.Operations·Europe & US East CoastFounders Associate; GTM, Sales & OpsStep into the engine room of a fast-growing tech startup and help shape Medusa’s commercial future while learning how to market and scale a category defining commerce platform.Engineering·Europe & US East CoastTechnical Support EngineerAs a Technical Support Engineer, you will play a critical role in helping our users build world-class commerce experiencesEngineering·Europe 

In [8]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """ 
    ### SCRAPED TEXT FROM WEBISTE
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website git.
    Your job is to extract the job postings and return then in JSON format containing
    following keys: `role`,`experience`,`skills` and `description. 
    Only return the Valid JSON.
    ### VALID JSON (NO PREAMBLE):

    """
    
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer, Open-source',
  'experience': '',
  'skills': '',
  'description': 'As apart of our open-source team, you will help improve our commerce platform and framework tooling to benefit Medusa developers and merchants.'},
 {'role': 'Founders Associate; GTM, Sales & Ops',
  'experience': '',
  'skills': '',
  'description': 'Step into the engine room of a fast-growing tech startup and help shape Medusa’s commercial future while learning how to market and scale a category defining commerce platform.'},
 {'role': 'Technical Support Engineer',
  'experience': '',
  'skills': '',
  'description': 'As a Technical Support Engineer, you will play a critical role in helping our users build world-class commerce experiences'},
 {'role': 'Solutions Engineer',
  'experience': '',
  'skills': '',
  'description': 'As a Solutions Engineer, you will help Medusa users succeed and bring back learnings to the core team.'}]

In [10]:
type(json_res)

list